In [7]:
import os
import sys
sys.path.append(os.path.realpath('../..'))
import aoc
my_aoc = aoc.AdventOfCode(2018,21)

In [8]:
registers = [0, 0, 0, 0, 0, 0]
ops_map = {}
ops = {
    'addr': lambda a, b : registers[a] + registers[b],
    'addi': lambda a, b : registers[a] + b,
    'mulr': lambda a, b : registers[a] * registers[b],
    'muli': lambda a, b : registers[a] * b,
    'banr': lambda a, b : registers[a] & registers[b],
    'bani': lambda a, b : registers[a] & b,
    'borr': lambda a, b : registers[a] | registers[b],
    'bori': lambda a, b : registers[a] | b,
    'setr': lambda a, b : registers[a],
    'seti': lambda a, b : a,
    'gtir': lambda a, b : 1 if a > registers[b] else 0,
    'gtri': lambda a, b : 1 if registers[a] > b else 0,
    'gtrr': lambda a, b : 1 if registers[a] > registers[b] else 0,
    'eqir': lambda a, b : 1 if a == registers[b] else 0,
    'eqri': lambda a, b : 1 if registers[a] == b else 0,
    'eqrr': lambda a, b : 1 if registers[a] == registers[b] else 0
}

def parse_input(lines):
    ip = 0
    instructions = []
    for line in lines:
        if "#ip" in line:
            _, ip = line.split(' ')
            ip = int(ip)
            continue
        cmd, val_a, val_b, val_c = line.split(' ')
        instructions.append(
            {
                "op": cmd,
                "a": int(val_a),
                "b": int(val_b),
                "c": int(val_c)
            }
        )
    return ip, instructions
        

def execute(ip, instructions):
    instruction = instructions[registers[ip]]
    output = f"ip={registers[ip]} {registers} {instruction['op']}"
    output += f" {instruction['a']} {instruction['b']} {instruction['c']}"
    registers[instruction['c']] = ops[instruction['op']](instruction['a'], instruction['b'])
    output += f" {registers}"
    # if registers[ip] == 28:
    #     print(output)
    # Afterward, move to the next instruction by adding one to the instruction pointer,
    # even if the value in the instruction pointer was just updated by an instruction.
    registers[ip] += 1

In [9]:
input_text = """#ip 3
seti 123 0 5
bani 5 456 5
eqri 5 72 5
addr 5 3 3
seti 0 0 3
seti 0 3 5
bori 5 65536 4
seti 8858047 4 5
bani 4 255 2
addr 5 2 5
bani 5 16777215 5
muli 5 65899 5
bani 5 16777215 5
gtir 256 4 2
addr 2 3 3
addi 3 1 3
seti 27 5 3
seti 0 6 2
addi 2 1 1
muli 1 256 1
gtrr 1 4 1
addr 1 3 3
addi 3 1 3
seti 25 1 3
addi 2 1 2
seti 17 4 3
setr 2 1 4
seti 7 3 3
eqrr 5 0 2
addr 2 3 3
seti 5 2 3"""

In [10]:
def detect_loop_old(sequence_generator):
    seen = []
    for i, num in enumerate(sequence_generator):
        seen.append(str(num))
        last_5 = ','.join(seen[-5:])
        all_seen = ','.join(seen)
        print(last_5, all_seen.count(last_5))
        if all_seen.count(last_5) > 10:
            return "Loop detected"
        if i > 100000:
            return "Breaking"
    return "No loop detected"

def detect_loop(sequence_generator):
    seen = []
    loop_size = 0  # Variable to store the size of the loop
    
    for i, num in enumerate(sequence_generator):
        seen.append(str(num))
        all_seen = ','.join(seen)
        # Check for repeating patterns of various sizes
        for window_size in range(2, min(100, len(seen) // 2)):
            pattern = ','.join(seen[-window_size:])
            count = all_seen.count(pattern)
            if count > 1:  # Pattern repeats
                if window_size > loop_size:
                    loop_size = window_size
                
            # Stop searching if we find a pattern repeating more than 10 times
            if count > 1000:
                return True, int(pattern.split(',')[0])
        
        if i > 100000:
            return True, -1
    
    return False, i

def run_program(ip, instructions):
    while True:
        if registers[ptr] < 0 or registers[ptr] >= len(instructions):
            return 
        execute(ptr, instructions)
        yield(registers[ip])
        

registers = [0, 0, 0, 0, 0, 0]
ptr, program = parse_input(input_text.splitlines())
loop = True
# 11513432 was determined by watching the value of registers[5]
# the relevant instruction is 28: eqrr 5 0 2
# 11513431 + 1 = 11513432
counter = 11513432 - 1
if part == 2:
    counter = 7434231 - 1
while loop:
    counter += 1
    registers = [counter, 0, 0, 0, 0, 0]
    loop, retval = detect_loop(run_program(ptr, program))
    
print(f"Part 1: {counter}")
    

Part 1: 11513432


In [11]:
nums = list(range(10))
nums[5:]

[5, 6, 7, 8, 9]

In [12]:
registers = [0, 0, 0, 0, 0, 0]
ptr, program = parse_input(input_text.splitlines())
seen = set()
last_one = None
while True:
    if registers[ptr] < 0 or registers[ptr] >= len(program):
        break 
    if registers[ptr] == 28:
        if registers[5] not in seen:
            seen.add(registers[5])
            last_one = int(registers[5])
        else:
            print(f"Part 2: {last_one}")
            break
    execute(ptr, program)
